## Greedy

In [ ]:
from typing import List

def overlap_intervals(intervals: List[List[int]]) -> int:
    if not intervals:
        return 0
    intervals.sort(key=lambda x:x[1])
    res = 1
    end = intervals[0][1]
    for i in intervals[1:]:
        if i[0] < end:
            # overlapping, just skip 
            continue
        end = i[1]
        res += 1
    return len(intervals)- res

intervals = [[1,2],[2,3],[3,4],[1,3]] 
print(intervals)
print(overlap_intervals(intervals))

## Backtracking

In [ ]:
from itertools import product

class Sudoku: 
    
    def __init__(self, shape=9, grid=3, empty=".") -> None:
        self.shape = shape
        self.grid = grid
        self.empty = empty 
        self.digits = set([str(num) for num in range(1, shape + 1)]) # 1-9
    
    def solve(self, board: List[List[str]]) -> None: 
        # modify board in place 
        self.search(board)
        
    def is_valid_state(self, board):
        # check if it is a valid solution
        # validate all rows 
        for row in self.get_rows(board):
            if not set(row) == self.digits: 
                return False
        # validate all cols 
        for col in self.get_cols(board):
            if not set(col) == self.digits:
                return False
        # validate sub-boxes
        for grid in self.get_grids(board):
            if not set(grid) == self.digits:
                return False
        return True 
    
    def search(self, board):
        if self.is_valid_state(board):
            return True # found solution
        
        # find the next empty spot and take a guess
        for row_idx, row in enumerate(board):
            for col_idx, elm in enumerate(row):
                if elm == self.empty:
                    # find candidates to construct the next state
                    for candidate in self.get_candidates(board, row_idx, col_idx):
                        board[row_idx][col_idx] = candidate
                        # recurse on the modified board
                        is_solved = self.search(board)
                        if is_solved:
                            return True
                        else:
                            # undo the wrong guess and start anew
                            board[row_idx][col_idx] = self.empty
                    # exhausted all candidates
                    # but none solves the problem
                    return False
        
        # no empty spot
        return True
    
    def get_candidates(self, board, row, col): 
        used_digits = set()
        # remove digits used by the same row
        used_digits.update(self.get_kth_row(board, row))
        # remove digits used by the same col
        used_digits.update(self.get_kth_col(board, col))
        # remove digits used by the sub-box
        used_digits.update(self.get_grid_of_cell(board, row, col))
        used_digits -= set([self.empty])
        candidates = self.digits - used_digits
        return candidates
    
    # helper functions to retrieve rows, cols, and grids 
    def get_kth_row(self, board, k):
        return board[k]
    
    def get_rows(self, board):
        for row in range(self.shape):
            yield board[row]
            
    def get_kth_col(self, board, k):
        return [board[row][k] for row in range(self.shape)]
    
    def get_cols(self, board):
        for col in range(self.shape):
            cells = [board[row][col] for row in range(self.shape)] 
            yield cells
    
    def get_grid_of_cell(self, board, row, col):
        row = row // self.grid * self.grid
        col = col // self.grid * self.grid
        return [
            board[r][c] for r, c in 
            product(range(row, row + self.grid), range(col, col + self.grid))
        ]
    
    def get_grids(self, board):
        for row in range(0, self.shape, self.grid):
            for col in range(0, self.shape, self.grid):
                cells = [
                    board[r][c] for r, c in 
                    product(range(row, row + self.grid), range(col, col + self.grid))
                ]
                yield cells

sdk = Sudoku() 
board = [["5","3",".",".","7",".",".",".","."],
["6",".",".","1","9","5",".",".","."],
[".","9","8",".",".",".",".","6","."],
["8",".",".",".","6",".",".",".","3"],
["4",".",".","8",".","3",".",".","1"],
["7",".",".",".","2",".",".",".","6"],
[".","6",".",".",".",".","2","8","."],
[".",".",".","4","1","9",".",".","5"],
[".",".",".",".","8",".",".","7","9"]] 
print("Input:")
for row in sdk.get_rows(board):
    print(row)
sdk.solve(board)
print("Output:")
for row in sdk.get_rows(board):
    print(row)


## Dynamic Programming

In [ ]:
import timeit

# Fibonacci 
"""
Fibonacci sequence: 0,1,1,2,3,5,8, ...

when n = 1, fib(1) = 0
when n = 2, fib(2) = 1
when n > 2, fib(n) = fib(n-1) + fib(n-2)

Given a number N return the index value of Fibnonacci sequence  
"""
fib_run = 0 
fib_memo = {}

# Time Complexity: O(2^n)
def fib_recursive(n):
    global fib_run 
    fib_run += 1
    if (n == 1):
        return 0
    if (n == 2):
        return 1 
    return fib_recursive(n-1) + fib_recursive(n-2)

# Time Complexity: O(n)
def fib_recursive_memo(n): 
    global fib_run, fib_memo
    fib_run += 1
    # return directly from cache if found
    if (n in fib_memo): 
        return fib_memo[n]
    if (n == 1):
        return 0
    if (n == 2):
        return 1 
    # cache the result
    fib_memo[n] = fib_recursive_memo(n-1) + fib_recursive_memo(n-2)
    return fib_memo[n]

# Time Complexity: O(n)
def fib_dp(n): 
    dp_memo = {}
    dp_memo[0] = 0
    dp_memo[1] = 0 
    dp_memo[2] = 1
    for i in range(2, n+1, 1): 
        dp_memo[i] = dp_memo[i-1] + dp_memo[i-2]
    return dp_memo[n]

fib_setup = "from __main__ import fib_recursive, fib_recursive_memo, fib_dp, fib_run, fib_memo"

timer = timeit.Timer(stmt="fib_recursive(30)", setup=fib_setup) 
fib_run = 0 
print("!!! recursion !!!")
print("   n:", 30)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

print("")

timer = timeit.Timer(stmt="fib_recursive(36)", setup=fib_setup) 
fib_run = 0 
print("!!! recursion !!!")
print("   n:", 36)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

print("")

timer = timeit.Timer(stmt="fib_recursive(38)", setup=fib_setup) 
fib_run = 0 
print("!!! recursion !!!")
print("   n:", 38)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

print("")

timer = timeit.Timer(stmt="fib_recursive_memo(38)", setup=fib_setup) 
fib_run = 0 
fib_memo = {}
print("!!! memo !!!")
print("   n:", 38)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

print("")

timer = timeit.Timer(stmt="fib_dp(38)", setup=fib_setup) 
fib_run = 0 
print("!!! DP !!!")
print("   n:", 38)
print("time: %2.6fs" % timer.timeit(number=1))
print("run#:", "{:,}".format(fib_run))

In [ ]:
# 0-1 Knapsack Problem

# Returns the maximum value that
# can be put in a knapsack of
# capacity C
# time complexity: O(2^n)
# space complexity: O(1)
def knapsack1(w, v, n, C): 
    # base case 
    if n == 0 or C == 0:
        return 0
    # if weight of the nth item is more than C
    # then it cannot be included 
    if (w[n-1] > C): 
        return knapsack1(w, v, n-1, C)
    # return the max of two case: 
    # 1. nth item included 
    # 2. not included 
    else: 
        return max(v[n-1] + knapsack1(w, v, n-1, C - w[n-1]), knapsack1(w, v, n-1, C))

# time complexity: O(n * C)
# space complexity: O(n * C)
def knapsack2(w, v, n, C): 
    k = [[0 for x in range(C+1)] for x in range(n+1)]
    # build table k from bottom up 
    for i in range(n+1): 
        for c in range(C+1): 
            if i==0 or c==0: 
                k[i][c] = 0
            elif w[i-1] > c: 
                k[i][c] = k[i-1][c]
            else:
                k[i][c] = max(v[i-1] + k[i-1][c - w[i-1]], k[i-1][c])
    return k[n][C]

# time complexity: O(n * C)
# space complexity: O(2 * C)
def knapsack3(w, v, n, C): 
    # 2 rows only: i%2 
    k = [[0 for x in range(C+1)] for x in range(2)]
    # build table k from bottom up 
    for i in range(n+1): 
        for c in range(C+1): 
            if i==0 or c==0: 
                k[i % 2][c] = 0
            elif w[i-1] > c: 
                k[i % 2][c] = k[(i-1) % 2][c]
            else:
                k[i % 2][c] = max(v[i-1] + k[(i-1) % 2][c - w[i-1]], k[(i-1) % 2][c])
    return k[n % 2][C]

# time complexity: O(n * C)
# space complexity: O(C)
def knapsack4(w, v, n, C): 
    k = [0 for i in range(C+1)]
    for i in range(1, n+1):  
        # compute from the back (right to left) 
        for c in range(C, 0, -1):  
            if w[i-1] <= c:
                k[c] = max(v[i-1] + k[c - w[i-1]], k[c])
    return k[C]  
    


w = [10, 20, 30]
v = [60, 100, 120]
C = 50
n = len(v)
print("max value1:", knapsack1(w, v, n, C))
print("max value2:", knapsack2(w, v, n, C))
print("max value3:", knapsack3(w, v, n, C))
print("max value4:", knapsack4(w, v, n, C))

In [ ]:
# egg drop 

# time complexity: O(k*n^2)
# space complexity: O(k*n)
def drop_memo(k: int, n:int):

    memo = dict() 
    def drop_recursive(k, n) -> int:
        
        # base case
        if k == 1:
            return n
        if n == 0:
            return 0 
        
        # memorization 
        if (k, n) in memo:
            return memo[(k, n)]
        
        # try every possibility
        res = float('INF')
        for f in range(1, n+1): 
            res = min(res, max(drop_recursive(k-1, f-1), drop_recursive(k, n-f)) + 1) 
            # memorization
            memo[(k, n)] = res 
        return res
    
    return drop_recursive(k, n)

# time complexity: O(k*n^2)
# space complexity: O(k*n)
def drop_dp1(k: int, n:int): 
    drop = [[0 for j in range(k+1)] for i in range(n+1)]
    
    # base case: one egg, drop f times  
    for f in range(1, n+1): 
        drop[f][1] = f 
    
    # base case: one floor, drop 1 time 
    for e in range(1, k+1): 
        drop[1][e] = 1
        
    for f in range(2, n+1): 
        for e in range(2, k+1): 
            res = float('INF')
            # floors = f 
            # eggs = e 
            # try to drop on each floor i              
            for i in range(1, f):
                res = min(res, max(drop[i-1][e-1], drop[f-i][e]) + 1)
            # res is the min times for floors(n) and eggs(e)
            drop[f][e] = res
    
    return drop[n][k]
    
# time complexity: O(k*n)
# space complexity: O(k*n)
def drop_dp2(k: int, n:int): 
    
    floor = [[0 for i in range(n+1)] for j in range(k+1)]
    # floor[0][...] = 0
    # floor[...][0] = 0 
    
    m = 0
    while (floor[k][m] < n): 
        m += 1
        for e in range(1, k+1): 
            floor[e][m] = floor[e][m-1] + floor[e-1][m-1] + 1
    return m

k = 2
n = 100
print("  egg:", k)
print("floor:", n)
print(" memo:", drop_memo(k, n))
print("  dp1:", drop_dp1(k, n))
print("  dp2:", drop_dp2(k, n))